<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/RNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import platform
import tensorflow as tf
# Importing S&P Prices
print (pd.__version__)
print (tf.__version__)
print (platform.python_version())

standard_and_poor = [
    2924.590088,2923.429932,2925.51001,2901.610107,2885.570068,
    2884.429932,2880.340088,2785.679932,2728.370117,2767.129883,
    2750.790039,2809.919922,2809.209961,2768.780029,2767.780029,
    2755.879883,2740.689941,2656.100098,2705.570068,2658.689941,
    2641.25, 2682.629883]



0.22.0
1.12.0-rc2
3.6.6


SIMPLE RNN EXAMPLE

In [0]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()
n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [0]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)

In [0]:
learning_rate = 0.001
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [0]:
n_iterations = 1500
batchsize = 50
